In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import open_spiel.python.examples.ubc_dispatch as dispatch
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from sklearn.model_selection import ParameterGrid
from pathlib import Path
import torch.nn.functional as F
import torch
import pickle


In [3]:
game_dir = os.environ["CLOCK_AUCTION_CONFIG_DIR"]

In [7]:
sats_games = []
PREFIXES = [
    # 'oct16_encumbereddeviations',
    # 'sep20',
    # 'dec11_bc',
    # 'dec13b_bc',
    # 'jan19'
    'jan19boring'
]

for prefix in PREFIXES:
    sats_games += [f'{prefix}/' + os.path.basename(x).replace('.json','') for x in glob.glob(f'{game_dir}/{prefix}/*')]
    
sats_games = list(set(sats_games))

sats_games = [
        s for s in sats_games if ('hide' not in s) and '1000' in s and 'risk_averse' not in s\
    # and 'rho1' in s \
    #and 'jan12_1' in s
]


# sats_games = [s for s in sats_games if 'activity' not in s]


print(len(sats_games))
print(sats_games)

20
['jan19boring/jan19boring_1_base_dev1000_rho0_t3', 'jan19boring/jan19boring_3_base_dev1000_rho1_t3', 'jan19boring/jan19boring_4_base_dev1000_rho1_t3_tie_break', 'jan19boring/jan19boring_0_base_dev1000_rho0_t3_tie_break', 'jan19boring/jan19boring_4_base_dev1000_rho1_t3', 'jan19boring/jan19boring_0_base_dev1000_rho0_t3', 'jan19boring/jan19boring_0_base_dev1000_rho1_t3', 'jan19boring/jan19boring_3_base_dev1000_rho1_t3_tie_break', 'jan19boring/jan19boring_2_base_dev1000_rho0_t3_tie_break', 'jan19boring/jan19boring_3_base_dev1000_rho0_t3_tie_break', 'jan19boring/jan19boring_2_base_dev1000_rho0_t3', 'jan19boring/jan19boring_2_base_dev1000_rho1_t3', 'jan19boring/jan19boring_4_base_dev1000_rho0_t3_tie_break', 'jan19boring/jan19boring_0_base_dev1000_rho1_t3_tie_break', 'jan19boring/jan19boring_4_base_dev1000_rho0_t3', 'jan19boring/jan19boring_1_base_dev1000_rho0_t3_tie_break', 'jan19boring/jan19boring_1_base_dev1000_rho1_t3_tie_break', 'jan19boring/jan19boring_1_base_dev1000_rho1_t3', 'jan19

# For running the real dumb test case

In [8]:
pgrid = ParameterGrid([
    {
        # Defaults
        'game': [g for g in sats_games]# if 'hide' not in g and 'imperfect' not in g]
    },
])
print(len(pgrid))

20


In [9]:
# ### Just GOF CFR
# num_iters = 999_000_000
# time_limit_seconds = int(3600 * 1)
# for config in pgrid:
#     CONFIG = 'cfr' # The YML network config folder for the main jobs
#     N_SEEDS = 3 # Number of seeds of each experiment to launch
#     extra_name = ''
#     overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 1800 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 50 --time_limit_seconds {time_limit_seconds} --use_wandb true'      
#     dispatch.dispatch_experiments(CONFIG, base_job_name='jan18_cfr_v6_seeds_v3', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr', start_seed=200)

In [10]:
num_iters = 999_000_000
time_limit_seconds = int(3600 * 1)
for config in pgrid:
    CONFIG = 'cfr_10' # The YML network config folder for the main jobs
    N_SEEDS = 5 # Number of seeds of each experiment to launch
    extra_name = ''
    overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 99999999 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 250 --time_limit_seconds {time_limit_seconds} --use_wandb true'      
    dispatch.dispatch_experiments(CONFIG, base_job_name='jan20_boring', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

Submitted batch job 111276
Submitted batch job 111277
Submitted batch job 111278
Submitted batch job 111279
Submitted batch job 111280
Submitted batch job 111281
Submitted batch job 111282
Submitted batch job 111283
Submitted batch job 111284
Submitted batch job 111285
Submitted batch job 111286
Submitted batch job 111287
Submitted batch job 111288
Submitted batch job 111289
Submitted batch job 111290
Submitted batch job 111291
Submitted batch job 111292
Submitted batch job 111293
Submitted batch job 111294
Submitted batch job 111295
Submitted batch job 111296
Submitted batch job 111297
Submitted batch job 111298
Submitted batch job 111299
Submitted batch job 111300
Submitted batch job 111301
Submitted batch job 111302
Submitted batch job 111303
Submitted batch job 111304
Submitted batch job 111305
Submitted batch job 111306
Submitted batch job 111307
Submitted batch job 111308
Submitted batch job 111309
Submitted batch job 111310
Submitted batch job 111311
Submitted batch job 111312
S

In [ ]:
!squeue --format="%.18i %.90j %.2t .%.10M%.6D %R"

In [15]:
!squeue | wc -l

901


In [ ]:
def check_queue():
    !ssh borg.cs.ubc.ca '/opt/slurm/bin/squeue -u newmanne --format="%.18i %.90j %.2t .%.10M %.6D %R"'


In [ ]:
check_queue()

In [ ]:
!squeue | grep "may30" | wc -l

In [ ]:
!scancel -u ubuntu

In [ ]:
# !scancel  46891 46893

In [ ]:
!rm -rf /shared/outputs/minitest/
try:
    Experiment.objects.get(name='minitest').delete()
except:
    pass
dispatch.dispatch_experiments('feb2', base_job_name='minitest', game_name='tiny', overrides = '--br_portfolio_path feb2 --eval_every 500 --num_training_episodes 1_000 --br_overrides "--num_training_episodes 1_000" --eval_overrides "--num_samples 100"')

In [ ]:
BestResponse.objects.last()

In [ ]:
Experiment.objects.all()

In [ ]:
!sacct --starttime 2022-03-11 --format=User,JobID,Jobname,partition,state,time,start,end,elapsed,MaxRss,MaxVMSize,nnodes,ncpus,nodelist

In [ ]:
Evaluation.objects.last()

In [ ]:
os.system("ssh borg.cs.ubc.ca '/opt/slurm/bin/sbatch'")

In [ ]:
!squeue --format="%.18i %.65j %.2t %.10M %.6D %R" | grep "eval" | awk '{print $1}' | xargs scancel

In [ ]:
from compress_pickle import dumps, loads


In [ ]:
loads?

In [ ]:
from compress_pickle.compressers.registry import get_known_compressions

In [ ]:
get_known_compressions()

In [ ]:
x = EquilibriumSolverRunCheckpoint.objects.last().policy

In [ ]:
len(x)

In [ ]:
len(dumps(pickle.loads(x), compression='bz2'))

In [ ]:
len(pickle.dumps(pickle.loads(x)))

In [ ]:
EquilibriumSolverRunCheckpoint.objects.last().walltime

In [ ]:
EquilibriumSolverRunCheckpoint.objects.last().t

In [ ]:
EquilibriumSolverRunCheckpoint.objects.last().bestresponse_set.all()

In [ ]:
len(EquilibriumSolverRun.objects.last().equilibriumsolverruncheckpoint_set.all().first().policy)

In [ ]:
len(EquilibriumSolverRun.objects.last().equilibriumsolverruncheckpoint_set.all().first().bestresponse_set.all().first().model)

In [ ]:
len(EquilibriumSolverRun.objects.last().equilibriumsolverruncheckpoint_set.all().first().bestresponse_set.all().last().model)